# Web Scraping with BeautifulSoup
    
**Description:**
This web scraper is designed to collect data on Fyodor Dostoevsky's literary works, including the titles, publication years, and additional information. The scraper targets a specific website that contains a curated list of Dostoevsky's works along with publication details. Using Python and popular libraries such as BeautifulSoup and pandas, the script navigates through the HTML structure of the webpage, extracts relevant information, and organizes it into a structured DataFrame. The resulting dataset provides a comprehensive overview of Dostoevsky's works, facilitating further analysis and exploration within a Jupyter notebook environment.

The obtained dataset is not only organized within a Jupyter notebook but also seamlessly exported to a Google Sheet through the Google Sheets API. This integration enables real-time collaboration, storage, and further analysis of the collected data on the cloud.

**Dependencies:**

* beautifulsoup4: For parsing HTML content.
* pandas: For data manipulation and storage.
* requests: For fetching HTML content from the website.
    

**1. Web Scraping Script**

In [55]:
# Import libraries

from bs4 import BeautifulSoup
import requests

In [66]:
# Obtains the raw data from the url

url = 'https://www.addall.com/books-in-order/fyodor-dostoevsky/'

page = requests.get(url)

soup = BeautifulSoup(page.text, 'html.parser')

In [75]:
# Selecting the main divs that contain the categories, descriptions, images, and titles

category_of_books = soup.find('div', class_='entry-content')

images_of_books = soup.find('div', class_='img-card')

titles_images_of_books = soup.find_all('div', class_="img-a")

description_of_books = soup.find_all('div', class_="over-card")

In [80]:
books = {}
book_categories = []
book_titles = []
book_images=[]
book_images_titles=[]
book_descriptions_titles = []
book_details_descr={}
book_details_img={}

categories = category_of_books.find_all('h2')
titles = category_of_books.find_all('div', class_='list-card')

# Adding the categories of the categories div to a list. 
# This list will become the keys of the dictionary that will store the titles as its values

for category in categories:
    book_categories.append(category.text)
    
for title in titles:
    book_titles.append(title.text)
    
for i in range (len(book_titles)):
    books[book_categories[i]] = book_titles[i]
    
# Adding the titles in the descriptions div to a list.
# This list will become the keys of the dictionary that will store the descriptions as its values

for title in description_of_books:
    # Find the 'a' tag within each div
    title_of_desc = title.find('a')
    book_descriptions_titles.append(title_of_desc.text)
    
for description in description_of_books:
    # Find the 'p' tag within each div
    description = description.find('p')
    book_descriptions.append(description.text)

for i in range(len(book_descriptions_titles)):
    book_details_descr[book_descriptions_titles[i]] = book_descriptions[i]


**2. DataFrame creation**

In [81]:
import pandas as pd

In [97]:
# We have two dictionaries, so two datasets were created

df = pd.DataFrame.from_dict(books, orient='index')
df2 = pd.DataFrame.from_dict(book_details, orient = 'index')

# Because the dfs were created from dicts, the key will be used as index. We are avoiding that by reseting the index.
# This adds a numerated index columns
# After that, we proceed to add the headers for the new columns

df.reset_index(inplace=True)
df.columns = ['Category', 'Title']

df2.reset_index(inplace=True)
df2.columns=['Title', 'Description']

df

,Category,Title
0,Standalone Novels In Publication Order,\n\nThe Double (1846)\nPoor Folk / Poor People...
1,Short Story Collections In Publication Order,\n\nPoor Folk and Other Stories (1845)\nAn Hon...
2,Writer’s Diary Books In Publication Order,"\n\nA Writer’s Diary, Volume One, 1873-1876 (1..."
3,Non-Fiction Books In Publication Order,\n\nWinter Notes on Summer Impressions (1863)\...


**3. Data Cleansing**

In [98]:
# At the beginning of the Title rows there were escape character. This removes them
df['Title'] = df['Title'].str.split('\n\n').str[1].str.strip()
df

,Category,Title
0,Standalone Novels In Publication Order,The Double (1846)\nPoor Folk / Poor People (18...
1,Short Story Collections In Publication Order,Poor Folk and Other Stories (1845)\nAn Honest ...
2,Writer’s Diary Books In Publication Order,"A Writer’s Diary, Volume One, 1873-1876 (1886)..."
3,Non-Fiction Books In Publication Order,Winter Notes on Summer Impressions (1863)\nDos...


In [99]:
#All the titles of a particular category were in a single row.
#This puts each title in a separate row

df['Title'] = df['Title'].str.split('\n')
df = df.explode('Title')
df

,Category,Title
0,Standalone Novels In Publication Order,The Double (1846)
0,Standalone Novels In Publication Order,Poor Folk / Poor People (1846)
0,Standalone Novels In Publication Order,The Landlady (1847)
0,Standalone Novels In Publication Order,White Nights (1848)
0,Standalone Novels In Publication Order,Uncle’s Dream (1859)
0,Standalone Novels In Publication Order,The Insulted and Injured / Humiliated and Insu...
0,Standalone Novels In Publication Order,The House of the Dead / Notes from a Dead Hous...
0,Standalone Novels In Publication Order,Notes from the Underground (1864)
0,Standalone Novels In Publication Order,Crime and Punishment (1866)
0,Standalone Novels In Publication Order,The Gambler (1866)


In [100]:
# In the titles, the YoP were included. This creates a separate column for it.

df['YoP'] = df['Title'].str.extract(r'(\d{4})')

#Removes the extra parnthesis that remain from the previous transformation

df['Title'] = df['Title'].str.split('(').str[0].str.strip()

#Removes the alternative titles

df['Title'] = df['Title'].str.split(' /').str[0].str.strip()
df['Title'] = df['Title'].str.split(', 1').str[0].str.strip()

#Removes the alternative titles
df2['Title'] = df2['Title'].str.split(' /').str[0].str.strip()

# Merges the two dfs using the titles as common column

df = pd.merge(df, df2, how='left', on='Title')
df.reset_index(inplace=True)
df = df.drop('index', axis=1)
df

,Category,Title,YoP,Description
0,Standalone Novels In Publication Order,The Double,1846,At once a comic masterpiece and a penetrating ...
1,Standalone Novels In Publication Order,Poor Folk,1846,This novel brought its 24 year old author crit...
2,Standalone Novels In Publication Order,The Landlady,1847,NaN
3,Standalone Novels In Publication Order,White Nights,1848,NaN
4,Standalone Novels In Publication Order,Uncle’s Dream,1859,NaN
5,Standalone Novels In Publication Order,The Insulted and Injured,1861,"1915. Dostoevsky, Russian novelist, is conside..."
6,Standalone Novels In Publication Order,The House of the Dead,1862,Accused of political subversion as a young man...
7,Standalone Novels In Publication Order,Notes from the Underground,1864,‘Notes from Underground translated in English ...
8,Standalone Novels In Publication Order,Crime and Punishment,1866,"Crime and Punishment, by Fyodor Dostoevsky, is..."
9,Standalone Novels In Publication Order,The Gambler,1866,NaN


In [101]:
# Removing the 'In Publication Order' substring from the categories

df['Category'] = df['Category'].str.replace('In Publication Order', '').str.strip()
df

,Category,Title,YoP,Description
0,Standalone Novels,The Double,1846,At once a comic masterpiece and a penetrating ...
1,Standalone Novels,Poor Folk,1846,This novel brought its 24 year old author crit...
2,Standalone Novels,The Landlady,1847,NaN
3,Standalone Novels,White Nights,1848,NaN
4,Standalone Novels,Uncle’s Dream,1859,NaN
5,Standalone Novels,The Insulted and Injured,1861,"1915. Dostoevsky, Russian novelist, is conside..."
6,Standalone Novels,The House of the Dead,1862,Accused of political subversion as a young man...
7,Standalone Novels,Notes from the Underground,1864,‘Notes from Underground translated in English ...
8,Standalone Novels,Crime and Punishment,1866,"Crime and Punishment, by Fyodor Dostoevsky, is..."
9,Standalone Novels,The Gambler,1866,NaN


In [102]:
# Cleaning NaN values since the json encoder in python does not allow NaN values for parsing

df = df.fillna('No Description')
df

,Category,Title,YoP,Description
0,Standalone Novels,The Double,1846,At once a comic masterpiece and a penetrating ...
1,Standalone Novels,Poor Folk,1846,This novel brought its 24 year old author crit...
2,Standalone Novels,The Landlady,1847,No Description
3,Standalone Novels,White Nights,1848,No Description
4,Standalone Novels,Uncle’s Dream,1859,No Description
5,Standalone Novels,The Insulted and Injured,1861,"1915. Dostoevsky, Russian novelist, is conside..."
6,Standalone Novels,The House of the Dead,1862,Accused of political subversion as a young man...
7,Standalone Novels,Notes from the Underground,1864,‘Notes from Underground translated in English ...
8,Standalone Novels,Crime and Punishment,1866,"Crime and Punishment, by Fyodor Dostoevsky, is..."
9,Standalone Novels,The Gambler,1866,No Description


**4. Data Export**

In [103]:
# Importing libraries for Google Sheets API connection

import gspread
from oauth2client.service_account import ServiceAccountCredentials
import json

In [50]:
# Set up credentials
scope =['https://spreadsheets.google.com/feeds',
         'https://www.googleapis.com/auth/drive']

# For Google Sheets
credentials = ServiceAccountCredentials.from_json_keyfile_name(
    r'insert credential here', scope)
gc = gspread.authorize(credentials)

# For Google Drive
credentials_drive = ServiceAccountCredentials.from_json_keyfile_name(
    r'insert credential here', scope)
gc_drive = gspread.authorize(credentials_drive)


In [51]:
# Open the previouly created Google Spreadsheet

spreadsheet = gc.open('dostoievski_data')

In [52]:
# Getting the default sheet

sheet = spreadsheet.sheet1

# Updates the sheet with the dataframe data

sheet.update([df.columns.values.tolist()] + df.values.tolist())

C:\Users\Eliana\AppData\Local\Temp\ipykernel_2372\3928591322.py:6: DeprecationWarning: [Deprecated][in version 6.0.0]: Method signature's arguments 'range_name' and 'values' will change their order. We recommend using named arguments for minimal impact. In addition, the argument 'values' will be mandatory of type: 'List[List]'. (ex) Worksheet.update(values = [[]], range_name=) 
  sheet.update([df.columns.values.tolist()] + df.values.tolist())


{'spreadsheetId': '10ej6aJMZKJOQJWmN_u7ulj0X2M7wI4QuXHVQtTU88zs',
 'updatedRange': "'Hoja 1'!A1:D34",
 'updatedRows': 34,
 'updatedColumns': 4,
 'updatedCells': 136}

In [53]:
# Open the spreadsheet by title
spreadsheet_title = "dostoievski_data"
spreadsheet = gc.open(spreadsheet_title)

# Print the URL of the spreadsheet
print(f"Spreadsheet URL: {spreadsheet.url}")

Spreadsheet URL: https://docs.google.com/spreadsheets/d/10ej6aJMZKJOQJWmN_u7ulj0X2M7wI4QuXHVQtTU88zs


### Next Steps

Data is ready to use in Google Sheets. The next step is to load this data in our Extraction Tool (Fivetran in this case)